In [8]:
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from dotenv import load_dotenv
from professions_list import ALL_PROFESSIONS, PROFESSIONS_LIST
from hh_resume_parser_db import HHResumeParser
from clickhouse_driver import Client

In [2]:
load_dotenv()

CLICKHOUSE_HOST: str = os.getenv("CLICKHOUSE_HOST")
CLICKHOUSE_USER: str = os.getenv("CLICKHOUSE_USER")
CLICKHOUSE_PASSWORD: str = os.getenv("CLICKHOUSE_PASSWORD")
CLICKHOUSE_DATABASE: str = os.getenv("CLICKHOUSE_DATABASE")

In [3]:
clickhouse = Client(host=CLICKHOUSE_HOST, port='9000', user=CLICKHOUSE_USER, password=CLICKHOUSE_PASSWORD, database=CLICKHOUSE_DATABASE, settings={'use_numpy': True})

In [ ]:
# clickhouse.execut("""
# CREATE TABLE IF NOT EXISTS hh_resumes (
#     id String,
#     title Nullable(String),
#     url Nullable(String),
#     specialization Array(Nullable(String)),
#     last_company Nullable(String),
#     last_position Nullable(String),
#     last_experience_description Nullable(String),
#     last_company_experience_period Nullable(String),
#     skills Array(String),
#     education Array(String),
#     courses Array(String),
#     salary Nullable(String),
#     age Nullable(Int),
#     total_experience Nullable(String),
#     experience_months Nullable(Int),
#     location Nullable(String),
#     gender Nullable(String),
#     applicant_status Nullable(String),
#     search_query Nullable(String),
#     parsed_date DateTime
# ) ENGINE = MergeTree()
# ORDER BY (parsed_date, id);
# """)

In [4]:
def get_unloaded_position(clickhouse):
    loaded_position = clickhouse.query_dataframe("""
    select search_query
    from hh_resumes
    """)
    loaded_position = set(loaded_position['search_query'].tolist())
    unloaded_position = list(set(PROFESSIONS_LIST['it_tech']) - loaded_position)
    return sorted(unloaded_position)

In [5]:
unloaded_position = get_unloaded_position(clickhouse)

In [6]:
resume_parser = HHResumeParser(timeout=60, max_retries=5)

In [7]:
for prof in tqdm(unloaded_position):
    df_resumes = resume_parser.load_resumes(
        search_terms=[prof],
        areas=['Москва'],
        pages=250,
        items_on_page=20,
        delay=2
    )

    resume_parser.save_to_clickhouse(df_resumes, clickhouse)

Сбор резюме:  17%|█████████████████████████████▍                                                                                                                                                 | 42/250 [20:42<1:42:32, 29.58s/it, Резюме=792, Запрос=Game Developer, Статус=Нет резюме - остановка]
NumPy support is not implemented for Array(Nullable(String)). Using generic column
NumPy support is not implemented for Array(String). Using generic column
NumPy support is not implemented for Array(String). Using generic column
NumPy support is not implemented for Array(String). Using generic column
  5%|████████████▋                                                                                                                                                                                                                                                 | 1/20 [20:42<6:33:27, 1242.50s/it]

=== ИТОГИ ===
Всего собрано резюме: 792
Распределение по поисковым запросам:
  Game Developer: 792
Успешно сохранено 792 записей в ClickHouse таблицу hh_resumes



Сбор резюме:  12%|█████████████████████▌                                                                                                                                                        | 31/250 [14:36<1:43:10, 28.27s/it, Резюме=580, Запрос=PHP-разработчик, Статус=Нет резюме - остановка]
NumPy support is not implemented for Array(Nullable(String)). Using generic column
NumPy support is not implemented for Array(String). Using generic column
NumPy support is not implemented for Array(String). Using generic column
NumPy support is not implemented for Array(String). Using generic column
 10%|█████████████████████████▍                                                                                                                                                                                                                                    | 2/20 [35:18<5:08:07, 1027.07s/it]

=== ИТОГИ ===
Всего собрано резюме: 580
Распределение по поисковым запросам:
  PHP-разработчик: 580
Успешно сохранено 580 записей в ClickHouse таблицу hh_resumes



Сбор резюме:  30%|█████████████████████████████████████████████████▊                                                                                                                    | 75/250 [1:16:33<2:58:38, 61.25s/it, Резюме=1450, Запрос=Администратор баз да, Статус=Нет резюме - остановка]
Error on localhost:9000 ping: [Errno 32] Broken pipe
Connection was closed, reconnecting.
Error on socket shutdown: [Errno 57] Socket is not connected
NumPy support is not implemented for Array(Nullable(String)). Using generic column
NumPy support is not implemented for Array(String). Using generic column
NumPy support is not implemented for Array(String). Using generic column
NumPy support is not implemented for Array(String). Using generic column
 15%|█████████████████████████████████████▋                                                                                                                                                                                                                

...

=== ИТОГИ ===
Всего собрано резюме: 5000
Распределение по поисковым запросам:
  Системный администратор: 5000


 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌            | 19/20 [45:20:30<2:16:57, 8217.46s/it]

Успешно сохранено 5000 записей в ClickHouse таблицу hh_resumes



Сбор резюме:   0%| | 1/250 [00:00<00:00, 2597.09it/s, Резюме=0, Запрос=Тестировщик, Страница=1/250]
Сбор резюме:   1%| | 2/250 [16:20<33:47:19, 490.48s/it, Резюме=0, Запрос=Тестировщик, Страница=1/25
Сбор резюме:   1%| | 2/250 [16:20<33:47:19, 490.48s/it, Резюме=20, Запрос=Тестировщик, Страница=2/2
Сбор резюме:   1%| | 3/250 [16:52<20:32:28, 299.39s/it, Резюме=20, Запрос=Тестировщик, Страница=2/2
Сбор резюме:   1%| | 3/250 [16:52<20:32:28, 299.39s/it, Резюме=40, Запрос=Тестировщик, Страница=3/2
Сбор резюме:   2%| | 4/250 [17:24<13:37:12, 199.32s/it, Резюме=40, Запрос=Тестировщик, Страница=3/2
Сбор резюме:   2%| | 4/250 [17:24<13:37:12, 199.32s/it, Резюме=60, Запрос=Тестировщик, Страница=4/2
Сбор резюме:   2%| | 5/250 [30:28<27:24:09, 402.65s/it, Резюме=60, Запрос=Тестировщик, Страница=4/2
Сбор резюме:   2%| | 5/250 [30:28<27:24:09, 402.65s/it, Резюме=80, Запрос=Тестировщик, Страница=5/2
Сбор резюме:   2%| | 6/250 [46:40<40:06:18, 591.71s/it, Резюме=80, Запрос=Тестировщик, Страница=5/2

=== ИТОГИ ===
Всего собрано резюме: 1500
Распределение по поисковым запросам:
  Тестировщик: 1500
Успешно сохранено 1500 записей в ClickHouse таблицу hh_resumes


### 